<a href="https://colab.research.google.com/github/ayush572/Fake_News_Classification_LSTM/blob/main/Fake_News_Classification_Kaggle_Dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install opendatasets

In [2]:
import opendatasets as od
import pandas

od.download(
	"https://www.kaggle.com/c/fake-news/data#/")


Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: ayushdg1
Your Kaggle Key: ··········


100%|██████████| 46.5M/46.5M [00:00<00:00, 89.4MB/s]



Extracting archive ./fake-news/fake-news.zip to ./fake-news


In [3]:
import pandas as pd
df = pd.read_csv('fake-news/train.csv')
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [4]:
# dropping nan values
df = df.dropna()

In [5]:
# now removing the label as label is the output, so removing it in the X
X = df.drop('label', axis = 1)

In [6]:
# y contains only the output of the same
Y = df['label']

In [7]:
print(X.shape)
Y.shape

(18285, 4)


(18285,)

In [8]:
import tensorflow as tf

In [9]:
# importing those things that are necessary to use LSTM
from tensorflow.keras.layers import Embedding # to convert the one-hot encoded vectors to featurized vectors
from tensorflow.keras.preprocessing.sequence import pad_sequences # helps to make the sentences of equal length
from tensorflow.keras.models import Sequential # to create a sequential model
from tensorflow.keras.preprocessing.text import one_hot # to convert the words to one_hot encoded vectors
from tensorflow.keras.layers import LSTM, Dense # using of LSTM and Dense layers
# Dense layer is being used as this is a classification problem, so that's why we require the same

In [10]:
# voca size - it defines like say within how many words our sentences will like
vocab_size=5000

In [11]:
messages = X.copy()

In [12]:
messages.reset_index(inplace = True) # as we have dropped the nan values, so, it leaves gaps in the rows. So, to counter that, we do reset_index

In [13]:
import nltk
import re
from nltk.corpus import stopwords

In [14]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [15]:
# Dataset preprocessing
from nltk.stem.porter import PorterStemmer # for stemming
ps = PorterStemmer()
corpus = []
for i in range(len(messages)): # len(messages) -> length of X
  # print(i)
  review = re.sub('[^a-zA-Z]', ' ', messages['title'][i]) # means, replacing all the characters, apart from a-z and A-Z, with '' blank space
  review = review.lower()
  review = review.split() # splitting - default delimiter white space
  # now review will be having the list of words insisde that specific sentence

  stemmed_words = []
  for word in review:
    if word not in stopwords.words('english'):
      word = ps.stem(word)
      stemmed_words.append(word)
  review = stemmed_words
  review = ' '.join(review)
  corpus.append(review)

In [16]:
corpus

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag',
 'russian research discov secret nazi militari base treasur hunter arctic photo',
 'us offici see link trump russia',
 'ye paid govern troll social media blog forum websit',
 'major leagu soccer argentin find home success new york time',
 'well fargo chief abruptli step new york time',
 'anonym donor pay million releas everyon arrest dakota access pipelin',
 'fbi close hilla

In [17]:
onehot_repr = [one_hot(words, vocab_size) for words in corpus] # traversing through all the words/sentences in corpus
onehot_repr

[[727, 2228, 4102, 833, 3225, 3645, 526, 1667, 3189, 1240],
 [4008, 697, 3714, 3371, 4337, 3247, 1511],
 [1889, 445, 3695, 429],
 [2823, 2762, 1667, 1145, 3567, 674],
 [806, 4337, 4147, 1366, 1156, 1450, 4337, 2552, 2232, 3017],
 [445,
  2630,
  169,
  2486,
  1377,
  2555,
  1635,
  1802,
  3700,
  2439,
  2403,
  1820,
  2432,
  2963,
  1511],
 [2499, 1886, 1412, 1856, 1934, 2424, 3581, 3626, 1472, 2320, 88],
 [4049, 1490, 997, 3797, 190, 3809, 2555, 3990, 1472, 2320, 88],
 [3331, 1112, 2850, 829, 4754, 1065, 636, 1764, 2555, 1460],
 [3477, 4626, 1675, 3290, 4585, 2607, 33, 611],
 [1826, 2393, 4952, 4160, 2474, 2343, 4979, 2576, 4740, 3367, 3968],
 [1145, 1881, 3225, 1065, 2555, 190],
 [1991, 4418, 4678, 3198, 2431, 910, 2946, 4865, 549],
 [2365, 2679, 29, 1355, 1787, 3278, 1610, 1472, 2320, 88],
 [1255, 1100, 4170, 3972, 657, 1472, 2320, 88],
 [2385, 1335, 3998, 3330, 2370, 4600, 1067, 709, 1802, 2340],
 [336, 3149, 697],
 [1840, 1456, 4833, 2339, 2555, 3559, 3031, 1511],
 [4397, 16

In [18]:
# now as all the sentences length were different, so we use pad_sequences to convert the same to sentences of equal length
max_length_list = max(onehot_repr, key=len)
max_length = len(max_length_list)

print("List with maximum length:", max_length_list)
print("Maximum length:", max_length)


List with maximum length: [1121, 2552, 302, 63, 4954, 3817, 2180, 2246, 137, 3334, 1779, 3765, 2898, 3149, 94, 4874, 299, 2180, 1808, 1906, 3390, 673, 2898, 1624, 3913, 3392, 2224, 3346, 4181, 4754, 697, 904, 4954, 1145, 3760, 3151, 3530, 3448, 1666, 1808, 1899, 3013, 712, 4526, 374, 3126, 2179]
Maximum length: 47


In [19]:
# Pad sequences to the maximum length
padded_sequences = pad_sequences(onehot_repr, maxlen=max_length, padding='pre')

In [20]:
print(padded_sequences)

[[   0    0    0 ... 1667 3189 1240]
 [   0    0    0 ... 4337 3247 1511]
 [   0    0    0 ...  445 3695  429]
 ...
 [   0    0    0 ... 1472 2320   88]
 [   0    0    0 ... 2377 4744 4054]
 [   0    0    0 ... 1116 4856   24]]


In [21]:
padded_sequences[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,  727, 2228, 4102,  833, 3225, 3645,  526,
       1667, 3189, 1240], dtype=int32)

In [22]:
# now comes the model creation
embedding_vector_features = 300
model = Sequential()
model.add(Embedding(vocab_size, embedding_vector_features, input_length = max_length))
model.add(LSTM(100)) # one LSTM layer having 100 neurons
# now adding the Dense Layer, as it is a classification problem
model.add(Dense(1, activation = 'sigmoid')) # gives the probability of output being in class 1 or class 2
model.compile(loss='binary_crossentropy', optimizer = 'adam', metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 47, 300)           1500000   
                                                                 
 lstm (LSTM)                 (None, 100)               160400    
                                                                 
 dense (Dense)               (None, 1)                 101       
                                                                 
Total params: 1660501 (6.33 MB)
Trainable params: 1660501 (6.33 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [23]:
import numpy as np
y_final = np.array(Y)
X_final = np.array(padded_sequences)

In [24]:
y_final

array([1, 0, 1, ..., 0, 1, 1])

In [25]:
X_final

array([[   0,    0,    0, ..., 1667, 3189, 1240],
       [   0,    0,    0, ..., 4337, 3247, 1511],
       [   0,    0,    0, ...,  445, 3695,  429],
       ...,
       [   0,    0,    0, ..., 1472, 2320,   88],
       [   0,    0,    0, ..., 2377, 4744, 4054],
       [   0,    0,    0, ..., 1116, 4856,   24]], dtype=int32)

In [26]:
X_final.shape, y_final.shape

((18285, 47), (18285,))

In [27]:
from sklearn.model_selection import train_test_split

# by setting the random state,
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size = 0.33, random_state = 42)

In [28]:
# now comes the model training
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10, batch_size=64)

Epoch 1/10
192/192 [==============================] - 19s 76ms/step - loss: 0.2632 - accuracy: 0.8878 - val_loss: 0.1983 - val_accuracy: 0.9142
Epoch 2/10
192/192 [==============================] - 4s 21ms/step - loss: 0.1224 - accuracy: 0.9509 - val_loss: 0.1933 - val_accuracy: 0.9225
Epoch 3/10
192/192 [==============================] - 3s 15ms/step - loss: 0.0690 - accuracy: 0.9760 - val_loss: 0.2272 - val_accuracy: 0.9128
Epoch 4/10
192/192 [==============================] - 2s 10ms/step - loss: 0.0368 - accuracy: 0.9882 - val_loss: 0.2938 - val_accuracy: 0.9148
Epoch 5/10
192/192 [==============================] - 3s 13ms/step - loss: 0.0205 - accuracy: 0.9922 - val_loss: 0.3732 - val_accuracy: 0.9095
Epoch 6/10
192/192 [==============================] - 2s 9ms/step - loss: 0.0124 - accuracy: 0.9967 - val_loss: 0.4064 - val_accuracy: 0.9115
Epoch 7/10
192/192 [==============================] - 2s 11ms/step - loss: 0.0079 - accuracy: 0.9976 - val_loss: 0.4755 - val_accuracy: 0.9128

Performance Metrices and Accuracy

In [40]:
y_pred = (model.predict(X_test) > 0.5).astype("int32")

189/189 [==============================] - 1s 3ms/step


In [42]:
from sklearn.metrics import confusion_matrix

In [43]:
confusion_matrix(y_test, y_pred)

array([[3135,  284],
       [ 286, 2330]])

In [41]:
y_pred

array([[1],
       [0],
       [0],
       ...,
       [0],
       [1],
       [1]], dtype=int32)

In [44]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_pred)

0.9055509527754764